In [17]:
from helpers import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
articles, categories, links, paths_finished, paths_unfinished, paths_all, shortest_path_distance = load_data()
get_graphs(paths_all)

NameError: name 'get_graphs' is not defined

In [18]:
G_paths, G_links = get_graphs(paths_all)

KeyError: "['path_id'] not in index"